In [0]:
%run ../_utils

In [0]:
import datetime

In [0]:
# widgets para debugar
# e parametros para receber do 'orquestrador' da camada bronze
table_name = dbutils.widgets.get("table_name")
dataset_location = dbutils.widgets.get("dataset_location")


# Camada bronze

Na camada bronze, nenhuma limpeza ou regra de negócio devem ser aplicadas aos dados.

só vamos ler em parquet e salvar em delta.

Vamos também utilizar da tabela de controle para termos o milestone da ultima execução (aqui nao será utilizado de fato, mas é interessante justamenete para o UPSERT)

## 1 - leitura dos dados

In [0]:
parquet_location = f"/FileStore/parquet/brazilian_ecommerce/{dataset_location}"
target_location = f"/FilesStore/delta/brazilian_ecommerce/{dataset_location}/bronze"

df = spark.read.parquet(parquet_location)


## Save dataframe

In [0]:
updated_timestamp = None
try:
    save_dataframe(df, format_mode="delta", target_location=target_location)
    updated_timestamp = datetime.datetime.now()
    
except Exception as e:
    dbutils.notebook.exit(None)


## Create delta table

In [0]:
tb_name = f"olist_bronze.{table_name}"
print(f"table full name: {tb_name}")

In [0]:
create_table(tb_name, target_location)

In [0]:
# para sair do notebook
# tudo que rodar após isso será ignorado quando o notebook for executado via workflow

# retornar o timestamp de atualização
dbutils.notebook.exit(updated_timestamp)

In [0]:
%sql

select count(*) from olist_bronze.customers